In [ ]:
import timeimport pandas as pdfrom datetime import datetimefrom marketwatch import MarketWatch

In [ ]:
def calculate_ema(price_series, periods):
    return price_series.ewm(span=periods, adjust=False).mean()

In [ ]:
def trade_on_crossover(df, game_name, stock_symbol):
    short_term = df['EMA_20'].iloc[-1]
    long_term = df['EMA_80'].iloc[-1]
    prev_short_term = df['EMA_20'].iloc[-2]
    prev_long_term = df['EMA_80'].iloc[-2]
    if short_term > long_term and prev_short_term <= prev_long_term:
        marketwatch.buy(game_name, stock_symbol, 100)
    elif short_term < long_term and prev_short_term >= prev_long_term:
        marketwatch.sell(game_name, stock_symbol, 100)

In [ ]:
def trade_multiple_stocks(stock_symbols):
    price_data = {symbol: [] for symbol in stock_symbols}
    for _ in range(160):
        for symbol in stock_symbols:
            price = marketwatch.get_price(symbol)
            price_data[symbol].append(price)
            if len(price_data[symbol]) > 150:
                price_data[symbol].pop(0)
        
        for symbol in stock_symbols:
            df = pd.DataFrame(price_data[symbol], columns=['Close'])
            for period in [20, 80]:
                ema_column_name = f'EMA_{period}'
                df[ema_column_name] = calculate_ema(df['Close'], period)
            
            if len(df) >= 80:
                trade_on_crossover(df, game_name, symbol)
        
        time.sleep(60)

In [ ]:
username = 'Your MarketWatch Username'
password = 'Your MarketWatch Password'
game_name = 'Your Game Name'
marketwatch = MarketWatch(username, password)
today = datetime.today().weekday()
if today >= 5:
    print('Today is a weekend. Skipping trading.')
else:
    tech_stocks = ['AAPL', 'GOOGL', 'MSFT']
    health_stocks = ['JNJ', 'MRK', 'PFE']
    utility_stocks = ['NEE', 'DUK', 'D']
    all_stocks = tech_stocks + health_stocks + utility_stocks
    trade_multiple_stocks(all_stocks)